<a href="https://colab.research.google.com/github/siposroland/cvs_hw_3d/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download model


In [0]:
!wget https://github.com/siposroland/cvs_hw_3d/raw/master/model.pth

# Neural Network

In [0]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch import optim

# Convolutional module (Conv+ReLU+BatchNorm)
class Conv(nn.Module):
  def __init__(self, in_channels, channels,kernel_size=3, stride=1):   #kernel_size=3
     super(Conv, self).__init__()
     self.conv = nn.Conv2d(in_channels, channels, kernel_size, stride=stride, padding=kernel_size//2, bias=False)   #padding=1
     self.bn = nn.BatchNorm2d(channels)
  def forward(self,x):
     return self.bn(torch.relu(self.conv(x)))
  

class ConvNet(nn.Module):
  def __init__(self, base_channels=16, in_channels=3, num_classes=55):  #in_channels=4
      super(ConvNet, self).__init__()   
      
      self.c11 = Conv(in_channels, base_channels)
      self.c12 = Conv(base_channels, base_channels)
      self.d1 = Conv(base_channels, base_channels*2, stride=2)

      self.c21 = Conv(base_channels*2, base_channels*2)
      self.c22 = Conv(base_channels*2, base_channels*2)
      self.d2 = Conv(base_channels*2, base_channels*4, stride=2)
        
      self.c31 = Conv(base_channels*4, base_channels*4)
      self.c32 = Conv(base_channels*4, base_channels*4)
      self.d3 = Conv(base_channels*4, base_channels*8, stride=2)
        
      self.c41 = Conv(base_channels*8, base_channels*8)
      self.c42 = Conv(base_channels*8, base_channels*8)
      self.d4 = Conv(base_channels*8, base_channels*16, stride=2)
        
      self.c51 = Conv(base_channels*16, base_channels*16)
      self.c52 = Conv(base_channels*16, base_channels*16)
      self.d5 = Conv(base_channels*16, base_channels*32, stride=2)
      # Input image is 32x32 -> after 5 downscaling the activation map is 1x1
      
      # Classifier is a convolution that produces num_classes class scores
      self.classifier = nn.Conv2d(base_channels*32,num_classes, 1) #kernel_size=1

  def forward(self,x):
        # Class all the layers
        x = self.d1(self.c12(self.c11(x)))
        x = self.d2(self.c22(self.c21(x)))
        x = self.d3(self.c32(self.c31(x)))
        x = self.d4(self.c42(self.c41(x)))
        x = self.d5(self.c52(self.c51(x)))
        return torch.squeeze(self.classifier(x))


haveCuda = torch.cuda.is_available()

#DATA AUGMENTATION
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49139968, 0.48215827, 0.44653124),
                         (0.24703233, 0.24348505, 0.26158768))
     #transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)) # mean = 0 and standard-deviation = 1
])

transform = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    # Random perturbance of brightness, contrast and color
    transforms.ColorJitter(brightness=0.3,contrast=0.3,saturation=0.3,hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.49139968, 0.48215827, 0.44653124),
                         (0.24703233, 0.24348505, 0.26158768))
    #transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)) # mean = 0 and standard-deviation = 1
])

#DATASETS
#trainSet = torchvision.datasets.ImageFolder(root="/content/trafficSignsHW/trainFULL", transform=transform)
#testSet = torchvision.datasets.ImageFolder(root="/content/trafficSignsHW/testFULL", transform=transform_val) #is_valid_file=None

#trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=32, shuffle=True) #sampler overfitting kevés adatra bs=128
#testLoader = torch.utils.data.DataLoader(testSet, batch_size=32, shuffle=False)


def createNet():
    net = ConvNet()
    if haveCuda:
        net = net.cuda()
    return net

def createLoss():
    return nn.CrossEntropyLoss()

# create optimizer
def createOptimizer():
    return optim.SGD(net.parameters(), lr=1e-2, momentum=0.5)  # lr=1e-1, momentum=0.9,nesterov=True, weight_decay=1e-4

numEpoch =20

def createScheduler():
    return optim.lr_scheduler.CosineAnnealingLR(optimizer,numEpoch,eta_min=1e-2)


#Progress bar
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))



def train(epoch):

    # variables for loss
    running_loss = 0.0
    correct = 0.0
    total = 0

    # set the network to train (for batchnorm and dropout)
    net.train()

    # Create progress bar
    bar = display(progress(0, len(trainLoader)), display_id=True)

    for i, data in enumerate(trainLoader, 0):
        
        inputs, labels = data
        if haveCuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        # Forward
        outputs = net(inputs)
        # Loss
        loss = criterion(outputs, labels)  #loss = torch.nn.functional.nll_loss(outputs, labels)
        # Backpropagation
        loss.backward()
        # Gradient method
        optimizer.step()

        # Do not include these steps in the computational graph
        with torch.no_grad():
            # Accumulate loss
            running_loss += loss.item()
            # Get indices of the largest goodness values
            _, predicted = torch.max(outputs, 1)
            # Count how many of the predictions equal the labels
            correct += predicted.eq(labels).sum().item()
            # Accumulate number of total images seen
            total += labels.shape[0]   ####

        # Progress bar
        bar.update(progress(i+1, len(trainLoader)))

    # return loss and accuracy
    tr_loss = running_loss / i
    tr_corr = correct / total * 100
    print("Train epoch %d loss: %.3f correct: %.2f" % (epoch + 1, running_loss / i, tr_corr))
    return tr_loss,tr_corr

# Function for validating a single epch
def val(epoch):

    # variables for loss
    running_loss = 0.0
    correct = 0.0
    total = 0

    # set the network to eval  (for batchnorm and dropout)
    net.eval()

    # Create progress bar
    bar = display(progress(0, len(testLoader)), display_id=True)

    for i, data in enumerate(testLoader, 0):
        # get the inputs
        inputs, labels = data
        if haveCuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        # Do not include these steps in the computational graph
        with torch.no_grad():
            # Forward
            outputs = net(inputs)
            # Compute loss
            loss = criterion(outputs, labels)

            # Compute statistics, just like before
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.shape[0]

        bar.update(progress(i+1, len(testLoader)))

    # return loss and accuracy
    val_loss = running_loss / i
    val_corr = correct / total * 100
    print("Test epoch %d loss: %.3f correct: %.2f" % (epoch + 1, running_loss / i, val_corr))
    return val_loss,val_corr


# classifying image

In [0]:
transform = transforms.Compose([transforms.ToTensor()]) 

#Read model
model = torch.load("./model.pth")
model.eval()

def prepend(list, str): 
    list = [str + i for i in list] 
    return(list) 

def sorted_nicely( l ):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

def hf_DL_task(path_of_source_pickle,path_of_destination_pickle):
  # Define used arrays and variables
  namesRGB = []

  # Get folders
  myFolderList = sorted([f.path for f in os.scandir("/content/HW") if f.is_dir()])

  # Put all RGB images to an array from "gx" folders
  for element in myFolderList:
    
    # Get and sort list of objects
    actualRGB = (sorted_nicely(glob.glob1(element + "/rgb", "*.jpg")))
    
    # Add path to all images
    path = element.replace('/content/', '')
    actualRGB = prepend(actualRGB, path + '/rgb/')
    
    
    # Put names to the big global array
    namesRGB = namesRGB + actualRGB
  
  # Open source pickle what contains data from previous tasks
  file = open(path_of_source_pickle,'rb')
  ourPrediction = pickle.load(file)
  
  # Loop throught all images
  for photoIndex in range(len(namesRGB)):
    # Get the name of the actual RGB picture
    actualPic = namesRGB[photoIndex]

    for actObject in ourPrediction[actualPic]["objects"]:
      # Get center coordinates of the object's bounding box
      u = actObject[0]
      v = actObject[1]
      w = actObject[2]
      h = actObject[3]
      c = actObject[4]
      
      #if Class  name== 'traffic sign'
      if c==0:
        crop_image = actualPic[v-w//2:v+w//2, u-h//2:u+h//2]
        rs_img=cv2.resize(crop_image, (32,32), interpolation = cv2.INTER_AREA)
        rs_img_rgb= cv2.cvtColor(rs_img,cv2.COLOR_BGR2RGB)
        dl_img = transform(rs_img_rgb)  # Preprocess image
        dl_img = dl_img.unsqueeze(0)  # Add batch dimension
        if haveCuda:
          dl_img=dl_img.cuda()
        output = model(dl_img)  # Forward pass
        _, pred = torch.max(output, 0)  # Get predicted class
        
        #Put subclassName to our prediction
        actObject[5] = pred.item()
  
  # Save results to our predictions
  pickle.dump(ourPrediction, open(path_of_destination_pickle, 'wb'))